In [2]:
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'../')
from m_argo_data import read_argo_data
from m_NCEP_read import read_NCEP
from m_fonctions import watervapor
import numpy as np

In [3]:
rep_argo_data = '/Users/chemon/ARGO_NEW/NEW_LOCODOX/data_test/'
rep_NCEP_data = '/Users/chemon/ARGO_NEW/LOCODOX/DATA/LOCODOX_EXTERNAL_DATA/NCEP/'
num_float = '6902802'

In [4]:
ds_argo_inair, ds_argo_inwater,ds_argo_Sprof, optode_height = read_argo_data(num_float,rep_argo_data)
print(optode_height) 

/Users/chemon/miniconda3/envs/locodox_env/lib/python3.10/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/chemon/miniconda3/envs/locodox_env/lib/python3.10/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/chemon/miniconda3/envs/locodox_env/lib/python3.10/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/chemon/miniconda3/envs/locodox_env/lib/python3.10/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/chemon/miniconda3/envs/locodox_env/lib/python3.10/site-packages/xarray/coding/tim

[-0.3]


In [5]:
ds_argo_inair

<xarray.Dataset>
Dimensions:                              (N_MEASUREMENT: 1480, N_PARAM: 8,
                                          N_CYCLE: 159, N_CALIB_PARAM: 1,
                                          N_CALIB_JULD: 1, N_HISTORY: 3)
Coordinates:
    LATITUDE                             (N_MEASUREMENT) float64 59.62 ... 65.1
    LONGITUDE                            (N_MEASUREMENT) float64 -38.96 ... -...
Dimensions without coordinates: N_MEASUREMENT, N_PARAM, N_CYCLE, N_CALIB_PARAM,
                                N_CALIB_JULD, N_HISTORY
Data variables: (12/127)
    DATA_TYPE                            (N_MEASUREMENT) object 'Argo traject...
    FORMAT_VERSION                       (N_MEASUREMENT) object '3.2 ' ... '3...
    HANDBOOK_VERSION                     (N_MEASUREMENT) object '1.2 ' ... '1...
    REFERENCE_DATE_TIME                  (N_MEASUREMENT) datetime64[ns] 1950-...
    DATE_CREATION                        (N_MEASUREMENT) datetime64[ns] 2023-...
    DATE_UPDATE                          (N_MEASUREMENT) datetime64[ns] 2023-...
    ...                                   ...
    HISTORY_PARAMETER                    (N_MEASUREMENT, N_HISTORY) object ' ...
    HISTORY_PREVIOUS_VALUE               (N_MEASUREMENT, N_HISTORY) float32 n...
    HISTORY_INDEX_DIMENSION              (N_MEASUREMENT, N_HISTORY) object 'n...
    HISTORY_START_INDEX                  (N_MEASUREMENT, N_HISTORY) float64 n...
    HISTORY_STOP_INDEX                   (N_MEASUREMENT, N_HISTORY) float64 n...
    HISTORY_QCTEST                       (N_MEASUREMENT, N_HISTORY) object ' ...
Attributes:
    title:                        Argo float trajectory file
    institution:                  CORIOLIS
    source:                       Argo float
    history:                      2023-06-14T18:41:35Z creation; 2023-06-14T1...
    references:                   http://www.argodatamgt.org/Documentation
    user_manual_version:          3.4
    Conventions:                  Argo-3.2 CF-1.6
    featureType:                  trajectory
    decoder_version:              CODA_055r
    comment_on_resolution:        JULD and PRES variable resolutions depend o...
    comment_on_measurement_code:  Meaning of some specific measurement codes ...

In [6]:
ds_NCEP_air,ds_NCEP_slp,ds_NCEP_rhum = read_NCEP(rep_NCEP_data)

In [7]:
ds_NCEP_air

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, time: 15692)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2014-01-01 ... 2024-09-27T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(1460, 73, 144), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis (2014)
    history:        created 2013/12 by Hoop (netCDF2.3)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  These...
    platform:       Model
    References:     http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reana...
    dataset_title:  NCEP-NCAR Reanalysis 1

In [8]:
print(ds_argo_inair['JULD'][0].values)
print(ds_NCEP_air['time'][8000].values)

2018-07-05T03:39:45.000000000
2019-06-24T00:00:00.000000000


In [9]:
# Interpolation 3D des variables NCEP sur les dates/positions InAIR et InWater
ds_NCEP_air2 = ds_NCEP_air.dropna("time")
ds_NCEP_air2

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, time: 15692)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2014-01-01 ... 2024-09-27T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(1460, 73, 144), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis (2014)
    history:        created 2013/12 by Hoop (netCDF2.3)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  These...
    platform:       Model
    References:     http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reana...
    dataset_title:  NCEP-NCAR Reanalysis 1

In [10]:
print(ds_argo_inair['LATITUDE'].values)
print(ds_argo_inair['LONGITUDE'].values)
print(ds_argo_inair['JULD'].values)


[59.62365167 59.62365167 59.62365167 ... 65.10297667 65.10297667
 65.10297667]
[-38.955295   -38.955295   -38.955295   ... -57.33742167 -57.33742167
 -57.33742167]
['2018-07-05T03:39:45.000000000' '2018-07-05T03:40:15.000000000'
 '2018-07-05T03:40:44.999999744' ... '2022-10-24T07:23:08.000000000'
 '2022-10-24T07:23:38.000000000' '2022-10-24T07:24:07.999999744']


In [17]:
import xarray as xr
ds_NCEP_air2 = ds_NCEP_air.copy()  # pour éviter de modifier l'original
if 'lon' in ds_NCEP_air2.coords:  # Vérifie que la coordonnée 'lon' existe
        ds_NCEP_air2['lon'] = xr.where(ds_NCEP_air['lon'] > 180, ds_NCEP_air['lon'] - 360, ds_NCEP_air['lon'])
print(ds_NCEP_air['lon'].values)
print(ds_NCEP_air2['lon'].values)

[  0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90.   92.5  95.   97.5 100.  102.5 105.  107.5 110.  112.5 115.  117.5
 120.  122.5 125.  127.5 130.  132.5 135.  137.5 140.  142.5 145.  147.5
 150.  152.5 155.  157.5 160.  162.5 165.  167.5 170.  172.5 175.  177.5
 180.  182.5 185.  187.5 190.  192.5 195.  197.5 200.  202.5 205.  207.5
 210.  212.5 215.  217.5 220.  222.5 225.  227.5 230.  232.5 235.  237.5
 240.  242.5 245.  247.5 250.  252.5 255.  257.5 260.  262.5 265.  267.5
 270.  272.5 275.  277.5 280.  282.5 285.  287.5 290.  292.5 295.  297.5
 300.  302.5 305.  307.5 310.  312.5 315.  317.5 320.  322.5 325.  327.5
 330.  332.5 335.  337.5 340.  342.5 345.  347.5 350.  352.5 355.  357.5]
[   0.     2.5    5.     7.5   10.    12.5   15.    17.5   20.    22.5
   25.    27.5   30.    32.5   35.    37.5   40.    

In [19]:
toto = ds_NCEP_air2['air'].interp(lat=ds_argo_inair['LATITUDE'],lon=ds_argo_inair['LONGITUDE'],time=ds_argo_inair['JULD'])
print(toto.values)

[280.27437283 280.2741369  280.27390097 ... 276.56670754 276.56718883
 276.56767012]


In [13]:
ds_NCEP_air['lon']

<xarray.DataArray 'lon' (lon: 144)>
array([  0. ,   2.5,   5. ,   7.5,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,
        25. ,  27.5,  30. ,  32.5,  35. ,  37.5,  40. ,  42.5,  45. ,  47.5,
        50. ,  52.5,  55. ,  57.5,  60. ,  62.5,  65. ,  67.5,  70. ,  72.5,
        75. ,  77.5,  80. ,  82.5,  85. ,  87.5,  90. ,  92.5,  95. ,  97.5,
       100. , 102.5, 105. , 107.5, 110. , 112.5, 115. , 117.5, 120. , 122.5,
       125. , 127.5, 130. , 132.5, 135. , 137.5, 140. , 142.5, 145. , 147.5,
       150. , 152.5, 155. , 157.5, 160. , 162.5, 165. , 167.5, 170. , 172.5,
       175. , 177.5, 180. , 182.5, 185. , 187.5, 190. , 192.5, 195. , 197.5,
       200. , 202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5,
       250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5,
       275. , 277.5, 280. , 282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5,
       300. , 302.5, 305. , 307.5, 310. , 312.5, 315. , 317.5, 320. , 322.5,
       325. , 327.5, 330. , 332.5, 335. , 337.5, 340. , 342.5, 345. , 347.5,
       350. , 352.5, 355. , 357.5], dtype=float32)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    units:          degrees_east
    long_name:      Longitude
    actual_range:   [  0.  357.5]
    standard_name:  longitude
    axis:           X

In [23]:
print(toto.values)

[nan nan nan ... nan nan nan]


In [55]:
ds_NCEP_air,ds_NCEP_slp,ds_NCEP_rhum = read_NCEP(rep_NCEP_data)
#toto=watervapor(ds_argo_inwater['TEMP_ADJUSTED'].values,ds_argo_inwater['PSAL_ADJUSTED'].values)
#toto2=(np.exp(24.4543-(67.4509*(100/(20+273.15)))-(4.8489*np.log(((273.15+20)/100)))-0.000544*35))
#print(toto2)
toto=watervapor(20,35)
type(toto)

NoneType

In [54]:
from m_fonctions import watervapor
toto=watervapor(20,35)
print(toto)

None


In [39]:
help(watervapor)

Help on function watervapor in module m_fonctions:

watervapor(T, S)



In [40]:
ds_NCEP_air

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, time: 15692)
Coordinates:
  * lat      (lat) float32 90.0 87.5 85.0 82.5 80.0 ... -82.5 -85.0 -87.5 -90.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * time     (time) datetime64[ns] 2014-01-01 ... 2024-09-27T18:00:00
Data variables:
    air      (time, lat, lon) float32 dask.array<chunksize=(1460, 73, 144), meta=np.ndarray>
Attributes:
    Conventions:    COARDS
    title:          4x daily NMC reanalysis (2014)
    history:        created 2013/12 by Hoop (netCDF2.3)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  These...
    platform:       Model
    References:     http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reana...
    dataset_title:  NCEP-NCAR Reanalysis 1